In [1]:
! pip install git+https://github.com/openai/whisper.git -q

     |████████████████████████████████| 4.9 MB 33.9 MB/s 
     |████████████████████████████████| 120 kB 73.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.5 MB/s 


In [2]:
! pip install gradio -q

     |████████████████████████████████| 5.3 MB 32.8 MB/s 
     |████████████████████████████████| 57 kB 6.5 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 270 kB 73.4 MB/s 
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 112 kB 79.3 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 2.3 MB 55.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 68 kB 5.4 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 594 kB 53.9 MB/s 
     |████████████████████████████████| 4.0 MB 65.1 MB/s 
     |████████████████████████████████| 856 kB 10.2 MB/s 


In [3]:
import gradio as gr 
import os
import sys
import subprocess
#from moviepy.editor import VideoFileClip


In [4]:
import whisper

model = whisper.load_model("medium")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 108MiB/s]


In [68]:
def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


In [143]:
#input_video = 'this+invention+makes+getting+out+of+bed+so+much+easier..mp4'

input_video = 'tamil_shorts.mp4'

In [144]:
audio_file = video2mp3(input_video)

In [145]:
from IPython.display import Audio
Audio(audio_file)

In [146]:
def translate(audio):
    
    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)
    return result




In [147]:
result = translate(audio_file)

Detected language: tamil


In [148]:
print(result["text"])

 Do you always have this button on? That's why they have given it to me. Oh! You look very disciplined. What time do you get up in the morning? 4.30. Will you pack milk instead of going home? No, I will study. You will study. But you look beautiful. Thank you.


In [155]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(input_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [156]:
from whisper.utils import write_vtt

In [157]:
output_dir = '/content/'
audio_path = audio_file.split(".")[0]

In [158]:
with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
    write_vtt(result["segments"], file=vtt)


In [159]:
subtitle = audio_path + ".vtt"
output_video = audio_path + "_subtitled.mp4"

In [160]:
os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")


# subprocess.call(["ffmpeg", "-i", input_video , "-vf", f"subtitles={subtitle}", f"{output_video}"], 
#                 stdout=subprocess.DEVNULL,
#                 stderr=subprocess.STDOUT)

0

In [161]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:

import gradio as gr 
import os
import sys
import subprocess
#from moviepy.editor import VideoFileClip

import whisper
from whisper.utils import write_vtt

model = whisper.load_model("medium")

def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


def translate(input_video):

    audio_file = video2mp3(input_video)
    
    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)

    output_dir = '/content/'
    audio_path = audio_file.split(".")[0]

    with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
      write_vtt(result["segments"], file=vtt)

    subtitle = audio_path + ".vtt"
    output_video = audio_path + "_subtitled.mp4"

    os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")

    return output_video



title = "Add Text/Caption to your YouTube Shorts - MultiLingual"

block = gr.Blocks()

with block:

    with gr.Group():
        with gr.Box(): 

           

            with gr.Row().style():
               
                inp_video = gr.Video(
                    label="Input Video",
                    type="filepath",
                    mirror_webcam = False
                )
                op_video = gr.Video()
            btn = gr.Button("Generate Subtitle Video")
        
        
        


        
        btn.click(translate, inputs=[inp_video], outputs=[op_video])
 
        gr.HTML('''
        <div class="footer">
                    <p>Model by <a href="https://github.com/openai/whisper" style="text-decoration: underline;" target="_blank">OpenAI</a> - Gradio App by <a href="https://twitter.com/1littlecoder" style="text-decoration: underline;" target="_blank">1littlecoder</a>
                    </p>
        </div>
        ''')

block.launch(debug = True)


/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://23847.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Detected language: english
Detected language: tamil
